In [1]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2022-11-17 08:19:04--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2022-11-17 08:19:04--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 14.6M 4s
    50K .......... .......... .......... .......... ..........  0%

In [2]:
%%bash
conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    requests-2.28.1            |   py38h06a4308_0          92 KB
    conda-package-handling-1.8.1|   py38h7f8727e_0         970 KB
    ld_impl_linux-64-2.38      |       h1181459_1         732 KB
    tk-8.6.12                  |       h1ccaba5_0         3.3 MB
    xz-5.2.5                   |       h7f8727e_1         389 KB
    _libgcc_mutex-0.1          |             main           3 KB
    zlib-1.2.12                |       h7f8727e_2         130 KB
    sqlite-3.38.5              |       hc218d9a_0         1.5 MB
    toolz-0.12.0               |   py38h06a4308_0          99 KB
    charset-normalizer-2.0.4   |     pyhd3eb1b0_0          33 KB
    idna-3.4                   |   py38h06a4308_0  

requests-2.28.1      |   92 KB | ########## | 100% 
conda-package-handli |  970 KB | ########## | 100% 
ld_impl_linux-64-2.3 |  732 KB | ########## | 100% 
tk-8.6.12            |  3.3 MB | ########## | 100% 
xz-5.2.5             |  389 KB | ########## | 100% 
_libgcc_mutex-0.1    |    3 KB | ########## | 100% 
zlib-1.2.12          |  130 KB | ########## | 100% 
sqlite-3.38.5        |  1.5 MB | ########## | 100% 
toolz-0.12.0         |   99 KB | ########## | 100% 
charset-normalizer-2 |   33 KB | ########## | 100% 
idna-3.4             |  109 KB | ########## | 100% 
pysocks-1.7.1        |   31 KB | ########## | 100% 
pyopenssl-22.0.0     |   49 KB | ########## | 100% 
ca-certificates-2022 |  131 KB | ########## | 100% 
wheel-0.37.1         |   31 KB | ########## | 100% 
cffi-1.15.0          |  226 KB | ########## | 100% 
ruamel_yaml-0.15.100 |  270 KB | ########## | 100% 
yaml-0.2.5           |   87 KB | ########## | 100% 
certifi-2022.9.24    |  157 KB | ########## | 100% 
pip-22.2.2  

In [3]:
!conda update -n base -c defaults conda --yes

Solving environment: / - \ | / - \ | / - \ | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::six==1.11.0=py36h372c433_1
  - defaults/linux-64::asn1crypto==0.24.0=py36_0
  - defaults/linux-64::chardet==3.0.4=py36h0f667ec_1
/ - \ | done

# All requested packages already installed.

Retrieving notices: ...working... done


In [4]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.8/site-packages"))

In [5]:
import sys
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/usr/local/lib/python3.8/site-packages']

In [6]:
!conda init zsh

modified      /usr/local/condabin/conda
modified      /usr/local/bin/conda
modified      /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.8/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
modified      /root/.zshrc

==> For changes to take effect, close and re-open your current shell. <==



In [7]:
!conda create --name MS-TCT python=3.8 --yes

Solving environment: \ done

## Package Plan ##

  environment location: /usr/local/envs/MS-TCT

  added / updated specs:
    - python=3.8


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |             main           3 KB
    ca-certificates-2022.10.11 |       h06a4308_0         124 KB
    certifi-2022.9.24          |   py38h06a4308_0         154 KB
    ld_impl_linux-64-2.38      |       h1181459_1         654 KB
    openssl-1.1.1s             |       h7f8727e_0         3.6 MB
    pip-22.2.2                 |   py38h06a4308_0         2.3 MB
    setuptools-65.5.0          |   py38h06a4308_0         1.1 MB
    tk-8.6.12                  |       h1ccaba5_0         3.0 MB
    wheel-0.37.1               |     pyhd3eb1b0_0          33 KB
    ------------------------------------------------------------
                                           Total:        11.0 M

In [8]:
!source /usr/local/bin/activate MS-TCT

In [9]:
! source activate /usr/local/envs/MS-TCT

In [10]:
!conda activate MS-TCT


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [11]:
!conda env list

# conda environments:
#
base                     /usr/local
MS-TCT                   /usr/local/envs/MS-TCT



In [12]:
!conda info --envs

# conda environments:
#
base                     /usr/local
MS-TCT                   /usr/local/envs/MS-TCT



In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [14]:
%cd /content/drive/MyDrive/MS-TCT-main

/content/drive/MyDrive/MS-TCT-main


#feature extraction


In [ ]:
!pwd

/content/drive/MyDrive/MS-TCT-main


In [ ]:
!git clone https://github.com/piergiaj/pytorch-i3d.git

fatal: destination path 'pytorch-i3d' already exists and is not an empty directory.


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
!conda install -c conda-forge h5py -y
!conda install -c conda-forge opencv -y


CondaError: KeyboardInterrupt

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ failed with initial frozen solve. Retrying with flexible solve.

CondaError: KeyboardInterrupt



In [ ]:
import cv2

In [ ]:
!conda install pytorch==0.3.0 -y

In [ ]:
!python3 /content/drive/MyDrive/MS-TCT-main/pytorch-i3d/extract_features.py \
-mode rgb \
-load_model /content/drive/MyDrive/MS-TCT-main/pytorch-i3d/models/rgb_charades.pt \
-root /content/drive/MyDrive/MS-TCT-main/charades \
-gpu 0\
-save_dir /content/drive/MyDrive/MS-TCT-main/feature_extract

[tensor([[[[[ 0.7882,  0.7961,  0.8039,  ...,  0.4824,  0.4745,  0.4745],
           [ 0.7882,  0.7961,  0.7961,  ...,  0.4824,  0.4745,  0.4745],
           [ 0.7804,  0.7882,  0.7961,  ...,  0.4824,  0.4745,  0.4745],
           ...,
           [-0.6157, -0.6078, -0.6157,  ...,  0.0824,  0.0667,  0.0588],
           [-0.6235, -0.6078, -0.6157,  ...,  0.1216,  0.1059,  0.0902],
           [-0.6392, -0.6157, -0.6157,  ...,  0.1686,  0.1373,  0.1216]],

          [[ 0.7882,  0.7961,  0.8039,  ...,  0.4824,  0.4745,  0.4745],
           [ 0.7882,  0.7961,  0.7961,  ...,  0.4824,  0.4745,  0.4745],
           [ 0.7804,  0.7882,  0.7961,  ...,  0.4824,  0.4745,  0.4745],
           ...,
           [-0.6157, -0.6078, -0.6157,  ...,  0.0824,  0.0667,  0.0588],
           [-0.6235, -0.6078, -0.6157,  ...,  0.1216,  0.1059,  0.0902],
           [-0.6392, -0.6157, -0.6157,  ...,  0.1686,  0.1373,  0.1216]],

          [[ 0.7882,  0.7961,  0.8039,  ...,  0.4824,  0.4745,  0.4745],
           [ 0

In [ ]:
import os
#path to frames
dirList = os.listdir("/content/drive/MyDrive/MS-TCT-main/charades")


#training

In [15]:
%%bash
pip install pytorch
pip install timm==0.4.12
pip install pickle5==0.0.9
pip install scikit-learn
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pytorch
Failed to build pytorch
  Running setup.py install for pytorch: started
  Running setup.py install for pytorch: finished with status 'error'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  error: subprocess-exited-with-error
  
  × Running setup.py install for pytorch did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> pytorch

note: This is an issue with the package mentioned above, not pip.
hint: See above for output from the failure.
tcmalloc: large alloc 1112711168 bytes == 0x37fc4000 @  0x7f153795b615 0x50960c 0x59c5e8 0x543ea4 0x4e6a2d 0x4c806e 0x4c64d5 0x4da73c 0x4c806e 0x4c64d5 0x4da73c 0x4c806e 0x4c64d5 0x4da73c 0x4c806e 0x4c64d5 0x4da73c 0x4c806e 0x4c64d5 0x4da73c 0x4c806e 0x4da6a6 0x4c806e 0x4

In [16]:
!python3.8 /content/drive/MyDrive/MS-TCT-main/train.py \
-dataset charades\
-mode rgb \
-model MS_TCT\
-train True \
-num_clips 256 \
-skip 0 \
-lr 0.0001 \
-comp_info False \
-epoch 50 \
-unisize True \
-alpha_l 1 \
-beta_l 0.05 \
-batch_size 32 


Random_SEED: 0
uni-size padd all T to 256
RGB mode /content/drive/MyDrive/MS-TCT-main/feature_extract
load data from /content/drive/MyDrive/MS-TCT-main/feature_extract
checking dataset
split!!!! training
100% 4/4 [00:01<00:00,  3.19it/s]
/usr/local/lib/python3.8/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
split!!!! testing
100% 4/4 [00:00<00:00,  4.60it/s]
you are processing MS_TCT
loaded False
/usr/local/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(r

#evaluation


In [ ]:
!python3.8 /content/drive/MyDrive/MS-TCT-main/Evaluation.py \
-pkl_path /content/drive/MyDrive/MS-TCT-main/save_logit/0.pkl

/content/drive/MyDrive/MS-TCT-main/apmeter.py:118: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  rg = torch.range(1, self.scores.size(0)).float()
/content/drive/MyDrive/MS-TCT-main/apmeter.py:145: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  ap[k] = precision[truth.byte()].sum() / max(truth.sum(), 1)
Test Frame-based map tensor(1.9844)
25 sampled Frame-based map tensor(2.0279)
APs for the classes tensor([ 95.2885,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,  

In [ ]:
!cd /content/drive/MyDrive/MS-TCT-main/best_logit/

In [ ]:
!pwd

/content/drive/MyDrive/MS-TCT-main


In [ ]:
import pickle

def load_pkl(pkl_name):
  with open(f'best_logit/{pkl_name}.pkl', 'rb') as f:
    data = pickle.load(f)

load_pkl(0)

ValueError: ignored